In [ ]:
using Distributions
#shortcut
flip(x) = Bernoulli(x)

A RV is a new type that is used for sampling graphical models.
It contains a distribution function, which can be called with rand, a name (optional, but can save bytes) and parents, whos value is important for the distgenerator function.

In [ ]:
type RV
    distgenerator::Function
    name #String or Char
    parents
end

In [ ]:
A = RV(() -> Normal(), "A", [])
B = RV((x)-> Normal(x,1), "B", [A])
C = RV((x)-> Normal(x,1), "C", [A])
D = RV((x,y)-> Normal(x, abs(y)), "D", [B,C])


W = RV(() -> Bernoulli(), 'W', [])
X = RV((x)-> Bernoulli(x), 'X', [W])
Y = RV((x)-> Bernoulli(x), 'Y', [W])
Z = RV((x,y)-> Binomial(int(x), int(y)), 'Z', [X,Y])

P = RV(()-> Poisson(4), "P", [])

The basic sampleMethods.
The parents are called recursive and sampled. It's values are needed for the distgenerator function of the RV for which the sampleRV function is called.<br>
sampleRV samples one or more RV's and returns the sampled values.<br>
It is also possible to sample with a trace, in that case, the return value is a dictionary with the RV's name or hash as key.

In [ ]:


function sampleRV(r::RV)
    # generate a single sample from scratch
    return sampleRV(r, Dict())
end

function sampleRV(r::RV, d::Dict{Any,Any})
    # generate a single sample but recycle already sampled values
    key = r.name
    if !haskey(d, key)
        vals   = map((s)->sampleRV(s,d), r.parents)
        d[key] = rand(r.distgenerator(vals...))
    end
    return d[key]
end

function sampleRV(rs::Array{RV,1}, d=Dict())
    # sample simultaneously from several random variables
    return [sampleRV(r,d) for r in rs]
end

#sample with a trace of needed variables. The parents are sampled too
#a dictionary is returned with all RV's hash as keys
function sampleRVTrace(r::RV)
    function f!(r::RV, d::Dict{Any,Any})
        key = hash(r)
        if !haskey(d, key)
            vals = map((s)->f!(s,d), r.parents)
            d[key] = rand(r.distgenerator(vals...))
        end
        return d[key]
    end
    d = Dict()
    f!(r, d)
    return d
end

#same as sampleRVTrace but instead, the RV.name is used
#this version allocates fewer bytes
function sampleRVNTrace(r::RV)
    function f!(r::RV, d::Dict{Any,Any})
        key = r.name
        if !haskey(d, key)
            vals = map((s)->f!(s,d), r.parents)
            d[key] = rand(r.distgenerator(vals...))
        end
        return d[key]
    end
    d = Dict()
    f!(r, d)
    return d
end

In [ ]:
sampleRV([A,D])

samples a given RV n times. An array is returned, which containes all sampled values as Float64

In [ ]:
function sampleRVTimes(r::RV, n :: Integer)
    result = Array(Float64,n)
    for i in 1:n
        result[i] = sampleRV(r)
    end
    result
end

a rejectionSampler that takes the maximum value of the samples as the envelope function.<br>
 The Rv is sampled n-times. An array with accepted values will be returned<br>
 M is a factor for tuning the envelope function c. <br>
It can be passed to the hist() function to see the PDF

In [ ]:
#sigma for variance
function rejectionSampler(r::RV, observed_value, sigma = 0.1 :: Float64, n=1000::Int64, trys=1000::Int64)
    f = sampleRVTimes(r,n)
    #g funktion 
    result = Float64[]
    #c * g(x) müssen immer >= f(x) sein
    i = 1
    while i < n
        if (f[i] - observed_value)^2 < sigma 
            push!(result,f[i])
        end
        i += 1
    end
    result
end

function sample_cond(r :: RV, observed_value, rvoi ::RV, sigma = 0.1 :: Float64, n=1000::Int64, trys=1000::Int64)
    result = Array(Float64,n)
    for j in 1 : n      
        for i in 1:trys
            s = sampleRV([r,rvoi])
            if abs(s[1] -observed_value) < sigma
                result[j] = s[2]
            end
        end
    end
    result
end

function sample_cond(r :: RV, observed_value, rvoi, sigma = 0.1 :: Float64, n=1000::Int64, trys=1000::Int64)
    result = Array(Array,n)
    for j in 1 : n      
        for i in 1:trys
            s = sampleRV([r,rvoi])
            if abs(s[1] -observed_value) < sigma
                result[j] = s[2:size(s)[1]]
            end
        end
    end
    result
end

In [ ]:
sample_cond(D,1,[A,B])